ASAP CRN Metadata validation 

# Team Rio. ASAP CRN Metadata validation 

28 Oct 2024
Andy Henrie


In [1]:
import pandas as pd
from pathlib import Path
import os, sys

sys.path.append(os.path.abspath((os.path.join(os.getcwd(), 'src/crn_utils'))))

from util import read_CDE, NULL, prep_table, read_meta_table
from validate import validate_table, ReportCollector
from update_schema import v1_to_v2, v2_to_v3_PMDBS, intervention_typer, create_upload_medadata_package
from checksums import get_md5_hashes, authenticate_with_service_account

%load_ext autoreload
%autoreload 2

root_path = Path.home() / ("Projects/ASAP/data/teams")


Streamlit NOT successfully imported


## CDEs
load the relavent CDEs

In [2]:
schema_version = "v1"
schema_path = Path.home() / "Projects/ASAP/crn-utils/resource/CDE"
CDEv1 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v2.1"
CDEv2 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v3.0"
CDEv3 = read_CDE(schema_version, local_path=schema_path)

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v1.csv
read local file
metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v2.1.csv
read local file
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v3.0.csv
read local file


## Clean Tables
write clean metadata tables according to CDE v3

### Team Rio

In [4]:
## convert 
team = "rio"
dataset_name = "familial-pd-scrna"

# skip source organization for now
# source = "ipsc"
root_path = Path.home() / ("Projects/ASAP/data/teams")

metadata_path = root_path / f"{team}/{dataset_name}/metadata/"
v3_path = metadata_path / "v3"
og_path = metadata_path / "og"  # this data is a copy of the lee/sn_pmdbs/metadata/v3

v3_meta_tables = ['STUDY', 'PROTOCOL', 'SAMPLE', 'DATA', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']



In [5]:
sheets = ["SAMPLE","STUDY","PROTOCOL","DATA"]

excel_path = og_path / "ASAP CRN Metadata table (SKM_scRNAseq).xlsx"
STUDY = pd.read_excel(excel_path,sheet_name="STUDY")#,header=1)#.drop(columns="Field")
SAMPLE = pd.read_excel(excel_path,sheet_name="SAMPLE")#,header=1)#.drop(columns="Field")
PROTOCOL = pd.read_excel(excel_path,sheet_name="PROTOCOL")#,header=1)#.drop(columns="Field")
DATA = pd.read_excel(excel_path,sheet_name="DATA")#,header=1)#.drop(columns="Field")

metadata_version = "v3.0"

METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"

In [6]:
STUDY

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCID,PI_google_scholar_id,DUA_version,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-RIO,ASAP_Hockemeyer/Bateup,Dissecting Genetic Interactions of Parkinson’s...,Team_Rio_DaN_scRNAseq,Team_Rio_DaN_scRNAseq_Familial_PD,Understanding the effects of familal PD gene m...,Dirk Hockemyer and Helen Bateup,hockemeyer@berkeley.edu; bateup@berkeley.edu,Khaja Mohieddin Syed; Jesse Dunnack,Khaja Mohieddin Syed,...,N/A ( its not published),NaN,Human embryonic stem cell (isogenic) derived i...,Dopamine Neurons,Dirk Hockemeyer_https://orcid.org/0000-0002-55...,NaN,NaN,NaN,NaN,NaN


In [7]:
# rationalize name
STUDY['team_dataset_id'] = dataset_name.replace("-", "_").replace(" ", "_")
STUDY


,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCID,PI_google_scholar_id,DUA_version,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-RIO,ASAP_Hockemeyer/Bateup,Dissecting Genetic Interactions of Parkinson’s...,familial_pd_scrna,Team_Rio_DaN_scRNAseq_Familial_PD,Understanding the effects of familal PD gene m...,Dirk Hockemyer and Helen Bateup,hockemeyer@berkeley.edu; bateup@berkeley.edu,Khaja Mohieddin Syed; Jesse Dunnack,Khaja Mohieddin Syed,...,N/A ( its not published),NaN,Human embryonic stem cell (isogenic) derived i...,Dopamine Neurons,Dirk Hockemeyer_https://orcid.org/0000-0002-55...,NaN,NaN,NaN,NaN,NaN


In [8]:
PROTOCOL

,sample_collection_summary,cell_extraction_summary,lib_prep_summary,data_processing_summary,github_url,protocols_io_DOI,other_reference
0,WIBR3 hESCs (passage 25-30) were maintained an...,Dopamine neurons were dissoaciated using acctu...,Standard 10X genomics chromium v3.1 3' single...,Standrard Cell ranger pipline from 10x genomci...,NaN,NaN,https://www.10xgenomics.com/support/single-cel...


In [9]:
SAMPLE

,Sample Name,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id,alternate_sample_id
0,DA_S1_B1,CMO303,SKM_GE1,NaN,NaN,NaN,B1,NaN,Brain,NaN,...,NaN,NaN,NaN,CL:2000097,NaN,Cell,NaN,NaN,NaN,NaN
1,DA_EWT3_B1,CMO304,SKM_GE1,NaN,NaN,NaN,B1,NaN,Brain,NaN,...,NaN,NaN,NaN,CL:2000097,NaN,Cell,NaN,NaN,NaN,NaN
2,DA_PRKN_H2-2_B1,CMO302,SKM_GE1,NaN,NaN,NaN,B1,NaN,Brain,NaN,...,NaN,NaN,NaN,CL:2000097,NaN,Cell,NaN,NaN,NaN,NaN
3,DA_SNCA/A30P_A2-3_B1,CMO301,SKM_GE1,NaN,NaN,NaN,B1,NaN,Brain,NaN,...,NaN,NaN,NaN,CL:2000097,NaN,Cell,NaN,NaN,NaN,NaN
4,DA_S2_B1,CMO303,SKM_GE2,NaN,NaN,NaN,B1,NaN,Brain,NaN,...,NaN,NaN,NaN,CL:2000097,NaN,Cell,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,DA_EWT8i_B5,CMO312,SKM_GE51,NaN,NaN,NaN,B5,NaN,Brain,NaN,...,NaN,NaN,NaN,CL:2000097,NaN,Cell,NaN,NaN,NaN,NaN
73,DA_S1_B5,CMO301,SKM_GE51,NaN,NaN,NaN,B5,NaN,Brain,NaN,...,NaN,NaN,NaN,CL:2000097,NaN,Cell,NaN,NaN,NaN,NaN
74,DA_S1i_B5,CMO302,SKM_GE51,NaN,NaN,NaN,B5,NaN,Brain,NaN,...,NaN,NaN,NaN,CL:2000097,NaN,Cell,NaN,NaN,NaN,NaN
75,DA_S3_B5,CMO303,SKM_GE51,NaN,NaN,NaN,B5,NaN,Brain,NaN,...,NaN,NaN,NaN,CL:2000097,NaN,Cell,NaN,NaN,NaN,NaN


file_description = "These are fastq files, GE refers to gene expression reads and CM refers to the cell multlplexing reads. For each library, both GE and CM reads are needed to get the cell assigned to particular sample with in the libraries. Here in the submission, all 10x libraries are made using cell plex multiplexing experiments. Number of samples within libraries varies and so cell number per library. Experiments are conducted in batchs and so WT samples are repeated as control for the resprcitive batch."

In [11]:
DATA['file_description'] = "NA"

In [12]:
# write the updated tables to the og folder with _ appended to the name
SAMPLE.to_csv(og_path / "SAMPLE_.csv")
DATA.to_csv(og_path / "DATA_.csv")
STUDY.to_csv(og_path / "STUDY_.csv")
PROTOCOL.to_csv(og_path / "PROTOCOL_.csv")


In [13]:
SAMPLE = read_meta_table(og_path / "SAMPLE_.csv")
DATA = read_meta_table(og_path / "DATA_.csv")
STUDY = read_meta_table(og_path / "STUDY_.csv")
PROTOCOL = read_meta_table(og_path / "PROTOCOL_.csv")


In [72]:
PROTOCOL

,sample_collection_summary,cell_extraction_summary,lib_prep_summary,data_processing_summary,github_url,protocols_io_DOI,other_reference
0,WIBR3 hESCs (passage 25-30) were maintained an...,Dopamine neurons were dissoaciated using acctu...,Standard 10X genomics chromium v3.1 3' single...,Standrard Cell ranger pipline from 10x genomci...,NA,NA,https://www.10xgenomics.com/support/single-cel...


In [18]:
SAMPLE

,Sample Name,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id,alternate_sample_id
0,DA_S1_B1,CMO303,SKM_GE1,NA,NA,NA,B1,NA,Brain,NA,...,NA,NA,NA,CL:2000097,NA,Cell,NA,NA,NA,NA
1,DA_EWT3_B1,CMO304,SKM_GE1,NA,NA,NA,B1,NA,Brain,NA,...,NA,NA,NA,CL:2000097,NA,Cell,NA,NA,NA,NA
2,DA_PRKN_H2-2_B1,CMO302,SKM_GE1,NA,NA,NA,B1,NA,Brain,NA,...,NA,NA,NA,CL:2000097,NA,Cell,NA,NA,NA,NA
3,DA_SNCA/A30P_A2-3_B1,CMO301,SKM_GE1,NA,NA,NA,B1,NA,Brain,NA,...,NA,NA,NA,CL:2000097,NA,Cell,NA,NA,NA,NA
4,DA_S2_B1,CMO303,SKM_GE2,NA,NA,NA,B1,NA,Brain,NA,...,NA,NA,NA,CL:2000097,NA,Cell,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,DA_EWT8i_B5,CMO312,SKM_GE51,NA,NA,NA,B5,NA,Brain,NA,...,NA,NA,NA,CL:2000097,NA,Cell,NA,NA,NA,NA
73,DA_S1_B5,CMO301,SKM_GE51,NA,NA,NA,B5,NA,Brain,NA,...,NA,NA,NA,CL:2000097,NA,Cell,NA,NA,NA,NA
74,DA_S1i_B5,CMO302,SKM_GE51,NA,NA,NA,B5,NA,Brain,NA,...,NA,NA,NA,CL:2000097,NA,Cell,NA,NA,NA,NA
75,DA_S3_B5,CMO303,SKM_GE51,NA,NA,NA,B5,NA,Brain,NA,...,NA,NA,NA,CL:2000097,NA,Cell,NA,NA,NA,NA


### infer the DATA and subject / sample from data


In [19]:
source = "ipsc"
bucket = f"asap-raw-team-{team}-{source}-{dataset_name}"
bucket = f"asap-raw-data-team-{team}" # for now old locations


key_file_path = Path.home() / f"Projects/ASAP/{team}-credentials.json"

res = authenticate_with_service_account(key_file_path)
print(res)


CompletedProcess(args='gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/rio-credentials.json', returncode=0, stdout='', stderr='Activated service account credentials for: [raw-admin-rio@dnastack-asap-parkinsons.iam.gserviceaccount.com]\n')
gsutil -u dnastack-asap-parkinsons hash -h gs://asap-raw-data-team-rio/rawdata/*.gz


The bucket is a mess... lets just do the md5 stuff by hand...

In [66]:

def extract_md5_from_details3(md5_file):
    md5s = {}
    with open(md5_file, "r") as f:
        lines = f.readlines()
        current_file = None
        for line in lines:
            if line.strip().startswith("Hashes [hex]"):
                current_file = line.strip().lstrip("Hashes [hex]: for ")
                # remove " for "
                # current_file = current_file.lstrip(" for")
                current_file = current_file.strip()
            elif "Hash (md5)" in line:
                md5s[current_file] = line.split(":")[1].strip()
            elif line.startswith("Hash (crc32c)"):
                pass
            else:
                print(f"cruff: {line.strip()}")
     
    return md5s

md5_file = root_path / f"{team}/all_gzip_hexhash.log"
bucket_files_md5 = extract_md5_from_details3( md5_file )

cruff: Hash (crc32c):		092e0757
cruff: Hash (crc32c):		1a832185
cruff: Hash (crc32c):		6455a0a5
cruff: Hash (crc32c):		a58debf7
cruff: Hash (crc32c):		827fe346
cruff: Hash (crc32c):		b7c7d827
cruff: Hash (crc32c):		15bea91e
cruff: Hash (crc32c):		43fd98eb
cruff: Hash (crc32c):		afd09f85
cruff: Hash (crc32c):		e0c4a8fd
cruff: Hash (crc32c):		53429736
cruff: Hash (crc32c):		242e15d1
cruff: Hash (crc32c):		9e37db6a
cruff: Hash (crc32c):		899bb803
cruff: Hash (crc32c):		edc0038b
cruff: Hash (crc32c):		6562d599
cruff: Hash (crc32c):		3512ea82
cruff: Hash (crc32c):		f1331100
cruff: Hash (crc32c):		24aa1cd0
cruff: Hash (crc32c):		ea9c3fdd
cruff: Hash (crc32c):		a920d30e
cruff: Hash (crc32c):		2de8992d
cruff: Hash (crc32c):		76792bb1
cruff: Hash (crc32c):		e78e4b84
cruff: Hash (crc32c):		5fa9320b
cruff: Hash (crc32c):		55d7624d
cruff: Hash (crc32c):		a3a0fe75
cruff: Hash (crc32c):		2edbce4e
cruff: Hash (crc32c):		e454e6b3
cruff: Hash (crc32c):		ad2e694e
cruff: Hash (crc32c):		643debd1
cruff: H

In [67]:

# prefix = "**/*.gz"
# bucket_files_md5 = get_md5_hashes( bucket, prefix)


In [68]:
df = pd.DataFrame(bucket_files_md5.items(), columns=["filename", "md5"])
df

,filename,md5
0,LRRK2_216_1_S37_L002_R1_001.fastq.gz:,27ba8eb1cdad532651c88f2b4fa46d06
1,LRRK2_216_1_S37_L002_R2_001.fastq.gz:,9ce5a0a4301e6483886d48076ea0b049
2,LRRK2_216_2_S38_L002_R1_001.fastq.gz:,758fff823e60c7a081a52fa2abc0555b
3,LRRK2_216_2_S38_L002_R2_001.fastq.gz:,90a6527c1d266a96d142a11f8c56e764
4,LRRK2_216_3_S39_L002_R1_001.fastq.gz:,442287bb42b21a93c3a38293eae2d517
...,...,...
105,wdata/SNCA_E_1_3_1_S6_L001_R2_001.fastq.gz:,40e5025c4aca745cdc54b4e5c8a9371e
106,wdata/SNCA_E_1_3_2_S5_L001_R1_001.fastq.gz:,d494475dc30991037991710ed3209c2e
107,wdata/SNCA_E_1_3_2_S5_L001_R2_001.fastq.gz:,cd536125104c29e0e8d111e6af9bf1b9
108,wdata/SNCA_E_1_3_3_S4_L001_R1_001.fastq.gz:,afe087123ddc0ef3d33d2fd4dc47336d


In [69]:
df['id'] = df['filename'].str.split("_").str[0:-3].str.join("_")
df

,filename,md5,id
0,LRRK2_216_1_S37_L002_R1_001.fastq.gz:,27ba8eb1cdad532651c88f2b4fa46d06,LRRK2_216_1_S37
1,LRRK2_216_1_S37_L002_R2_001.fastq.gz:,9ce5a0a4301e6483886d48076ea0b049,LRRK2_216_1_S37
2,LRRK2_216_2_S38_L002_R1_001.fastq.gz:,758fff823e60c7a081a52fa2abc0555b,LRRK2_216_2_S38
3,LRRK2_216_2_S38_L002_R2_001.fastq.gz:,90a6527c1d266a96d142a11f8c56e764,LRRK2_216_2_S38
4,LRRK2_216_3_S39_L002_R1_001.fastq.gz:,442287bb42b21a93c3a38293eae2d517,LRRK2_216_3_S39
...,...,...,...
105,wdata/SNCA_E_1_3_1_S6_L001_R2_001.fastq.gz:,40e5025c4aca745cdc54b4e5c8a9371e,wdata/SNCA_E_1_3_1_S6
106,wdata/SNCA_E_1_3_2_S5_L001_R1_001.fastq.gz:,d494475dc30991037991710ed3209c2e,wdata/SNCA_E_1_3_2_S5
107,wdata/SNCA_E_1_3_2_S5_L001_R2_001.fastq.gz:,cd536125104c29e0e8d111e6af9bf1b9,wdata/SNCA_E_1_3_2_S5
108,wdata/SNCA_E_1_3_3_S4_L001_R1_001.fastq.gz:,afe087123ddc0ef3d33d2fd4dc47336d,wdata/SNCA_E_1_3_3_S4


In [70]:
df['path'] = df['filename'].apply(lambda x: x.split("/")[0] if "/" in x else "")
df

,filename,md5,id,path
0,LRRK2_216_1_S37_L002_R1_001.fastq.gz:,27ba8eb1cdad532651c88f2b4fa46d06,LRRK2_216_1_S37,
1,LRRK2_216_1_S37_L002_R2_001.fastq.gz:,9ce5a0a4301e6483886d48076ea0b049,LRRK2_216_1_S37,
2,LRRK2_216_2_S38_L002_R1_001.fastq.gz:,758fff823e60c7a081a52fa2abc0555b,LRRK2_216_2_S38,
3,LRRK2_216_2_S38_L002_R2_001.fastq.gz:,90a6527c1d266a96d142a11f8c56e764,LRRK2_216_2_S38,
4,LRRK2_216_3_S39_L002_R1_001.fastq.gz:,442287bb42b21a93c3a38293eae2d517,LRRK2_216_3_S39,
...,...,...,...,...
105,wdata/SNCA_E_1_3_1_S6_L001_R2_001.fastq.gz:,40e5025c4aca745cdc54b4e5c8a9371e,wdata/SNCA_E_1_3_1_S6,wdata
106,wdata/SNCA_E_1_3_2_S5_L001_R1_001.fastq.gz:,d494475dc30991037991710ed3209c2e,wdata/SNCA_E_1_3_2_S5,wdata
107,wdata/SNCA_E_1_3_2_S5_L001_R2_001.fastq.gz:,cd536125104c29e0e8d111e6af9bf1b9,wdata/SNCA_E_1_3_2_S5,wdata
108,wdata/SNCA_E_1_3_3_S4_L001_R1_001.fastq.gz:,afe087123ddc0ef3d33d2fd4dc47336d,wdata/SNCA_E_1_3_3_S4,wdata


In [71]:
df['path'].unique()

array(['', 'SNCA_A53T_rawdata', 'wdata'], dtype=object)

In [64]:
df['just_fn'] = df['filename'].str.lstrip(df['path']).str.lstrip("/")

AttributeError: Can only use .str accessor with string values!

In [63]:
df['subject_id'] = df['id'].apply(lambda x: f"{x[-2:]}_{x[:-2]}")
df.head()

,filename,md5,id,path,subject_id
0,LRRK2_216_1_S37_L002_R1_001.fastq.gz:,27ba8eb1cdad532651c88f2b4fa46d06,LRRK2_216_1_S37,,37_LRRK2_216_1_S
1,LRRK2_216_1_S37_L002_R2_001.fastq.gz:,9ce5a0a4301e6483886d48076ea0b049,LRRK2_216_1_S37,,37_LRRK2_216_1_S
2,LRRK2_216_2_S38_L002_R1_001.fastq.gz:,758fff823e60c7a081a52fa2abc0555b,LRRK2_216_2_S38,,38_LRRK2_216_2_S
3,LRRK2_216_2_S38_L002_R2_001.fastq.gz:,90a6527c1d266a96d142a11f8c56e764,LRRK2_216_2_S38,,38_LRRK2_216_2_S
4,LRRK2_216_3_S39_L002_R1_001.fastq.gz:,442287bb42b21a93c3a38293eae2d517,LRRK2_216_3_S39,,39_LRRK2_216_3_S


In [17]:
in_tables = sheets

og_tables = {}
for table_name in in_tables:
    df = read_meta_table(f"{og_path}/{table_name}_.csv")
    og_tables[table_name] = df
og_tables.keys() 

dict_keys(['SAMPLE', 'STUDY', 'PROTOCOL', 'DATA'])

In [30]:
# assay table has sample_ids listed but in a subject_id column
og_tables['ASSAY_scRNAseq'].rename(columns={"subject_id": "sample_id"}, inplace=True)

In [31]:
CDE = CDEv3
dfs = {}
for table,df in og_tables.items():
    if table == "ASSAY_scRNAseq":
        table = "ASSAY_RNAseq" 

    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    dfs[table] = full_table

🚨⚠️❗ **Missing Required Fields in STUDY: number_samples, metadata_tables**
🚨⚠️❗ **7 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- DUA_version: 1/1 empty rows (REQUIRED)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.
🚨⚠️❗ **Extra field in STUDY: numbe_samples**

All required fields are present in *PROTOCOL* table.
No empty entries (NULL) found .
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **Missing Optional Fields in SUBJECT: primary_diagnosis_text**
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- race: 284/334 empty rows (REQUIRED)
No invalid entries found in Enum fields.
🚨⚠️❗ **Extra field in SUBJECT: organism**
🚨⚠️❗ **Extra field in SUBJECT:

### STUDY
add number_samples, metadata_tables

### PROTOCL
all good

### SUBJECT

move to SAMPLE
- organism

move to CLINPATH
-  AMPPD_id, GP2_id, ethnicity, family_history, last_diagnosis, age_at_onset, age_at_diagnosis, first_motor_symptom, hx_dementia_mci, hx_melanoma, education_level, smoking_status, APOE_e4_status, cognitive_status, time_from_baseline


### SAMPLE
import organism from SUBJECT


FIX : source_sample_id, replicate, replicate_count, repeated_sample, batch, organism, assay_type, condition_id, organism_ontology_term_id

source_sample_id <> source_subject_id


### DATA
okay


Move to ASSAY_RNAseq    
- technolog, omics, sequencing_end, sequencing_length, sequencing_instrument, molecular_source, assay

Not sure about these:
- R1_or_R2, technology_full_name, subject_ids, source_subject_ids


### CLINPATH
Import from SUBJECT
- AMPPD_id, GP2_id, ethnicity, family_history, last_diagnosis, age_at_onset, age_at_diagnosis, first_motor_symptom, hx_dementia_mci, hx_melanoma, education_level, smoking_status, smoking_years, APOE_e4_status, cognitive_status, time_from_baseline, path_autopsy_second_dx, path_autopsy_third_dx, path_autopsy_fourth_dx, path_autopsy_fifth_dx, path_autopsy_sixth_dx, path_autopsy_seventh_dx, path_autopsy_eight_dx, other_cause_death_1, other_cause_death_2, known_pathogenic_mutation, PD_pathogenic_mutation, path_mckeith, sn_neuronal_loss, path_infarcs, path_nia_ri, path_nia_aa_a, path_nia_aa_b, path_nia_aa_c, TDP43, arteriolosclerosis_severity_scale, amyloid_angiopathy_severity_scale, path_ad_level, dig_slide_avail, quant_path_avail


### ASSAY
Import from DATA
- technology, omic, molecular_source, input_cell_count, assay, sequencing_end, sequencing_length, sequencing_instrument**

Move to PMDBS (but need to map to samples)
- hemisphere, region_level_1, region_level_2, region_level_3, pm_PH


In [32]:
STUDY = dfs['STUDY']
PROTOCOL = dfs['PROTOCOL']
SUBJECT = dfs['SUBJECT']
SAMPLE = dfs['SAMPLE']
DATA = dfs['DATA']
CLINPATH = dfs['CLINPATH']
ASSAY_RNAseq = dfs['ASSAY_RNAseq']



This multiome data is complicated compared to other submissions as data are multiplexed across subjects.  Need to confirm with team where genetic information to assign cells to individual subjects can be found.

Also need to test the fidelity of running this multiome (ARC) assay results along with rna seq only data.  Should be kosher.

Current 'sample_id' encoding is inconsistent between SAMPLE and DATA tables.

In [33]:
from update_schema import filter_table_columns
    

In [26]:
# STUDY
STUDY_ = filter_table_columns(dfs["STUDY"], CDEv3, "STUDY")
metadata_version = "v3.0"
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"
STUDY_['metadata_version_date'] = METADATA_VERSION_DATE
STUDY_

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,number_samples,sample_types,types_of_samples,DUA_version,metadata_tables,PI_ORCID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-VOET,Aerts Lab,Understanding inherited and acquired genetic v...,sn_multi_seq,Aerts_singlecellmultiome_RNA,The functional roles of inherited and acquired...,Stein Aerts,stein.aerts@kuleuven.be,"Alexandra Pan?�kov�, Julie De Man, Gert Hulsel...",Gert Hulselmans,...,NA,"Anterior Cingulate Cortex, Mid Brain, Motor Co...",late-stage PD and control postmortem brains,NA,NA,0000-0002-8006-0315,NA,NA,v3.0_20241028,NA


In [35]:
# PROTOCOL
PROTOCOL_ = filter_table_columns(dfs["PROTOCOL"], CDEv3, "PROTOCOL")
PROTOCOL_

,sample_collection_summary,cell_extraction_summary,lib_prep_summary,data_processing_summary,github_url,protocols_io_DOI,other_reference
0,"This dataset contains samples from Mid Brain, ...",Nuclei were isolated using glass dounces and m...,Nuclei were processed using either: (i) 10x Ge...,Illumina sequencing reads were de-multiplexed ...,https://github.com/10XGenomics/cellranger,dx.doi.org/10.17504/protocols.io.n2bvj3yqnlk5/v1,https://www.parsebiosciences.com/products/ever...


In [36]:
# SUBJECT
SUBJECT_ = filter_table_columns(dfs["SUBJECT"], CDEv3, "SUBJECT")
SUBJECT_

,subject_id,source_subject_id,biobank_name,sex,age_at_collection,race,primary_diagnosis,primary_diagnosis_text
0,ASA_001A,SD050/19,Edinburgh_UK,Male,57.0,NA,Healthy Control,NA
1,ASA_001B,SD050/19,Edinburgh_UK,Male,57.0,NA,Healthy Control,NA
2,ASA_002A,SD041/19,Edinburgh_UK,Female,40.0,NA,Healthy Control,NA
3,ASA_002B,SD041/19,Edinburgh_UK,Female,40.0,NA,Healthy Control,NA
4,ASA_003A,SD026/19,Edinburgh_UK,Male,58.0,NA,Healthy Control,NA
...,...,...,...,...,...,...,...,...
329,ASA_188B,P65/19,QSBB_UK,Male,74.0,NA,Idiopathic PD,NA
330,ASA_189A,P81/19,QSBB_UK,Female,74.0,NA,Idiopathic PD,NA
331,ASA_189B,P81/19,QSBB_UK,Female,74.0,NA,Idiopathic PD,NA
332,ASA_190A,P87/19,QSBB_UK,Male,72.0,NA,Idiopathic PD,NA


In [44]:
dfs["SUBJECT"]['organism']

0      Human
1      Human
2      Human
3      Human
4      Human
       ...  
329    Human
330    Human
331    Human
332    Human
333    Human
Name: organism, Length: 334, dtype: object

In [64]:

# SAMPLE
SAMPLE_= filter_table_columns(dfs["SAMPLE"], CDEv3, "SAMPLE")
SAMPLE_


,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,organism,tissue,assay_type,...,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,assay_ontology_term_id,donor_id,pm_PH,cell_type_ontology_term_id,source_RIN,DV200,suspension_type
0,ASA_051B,ASA_051,NA,NA,NA,NA,NA,NA,Brain,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,ASA_052B,ASA_052,NA,NA,NA,NA,NA,NA,Brain,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,ASA_053B,ASA_053,NA,NA,NA,NA,NA,NA,Brain,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,ASA_054B,ASA_054,NA,NA,NA,NA,NA,NA,Brain,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,ASA_055B,ASA_055,NA,NA,NA,NA,NA,NA,Brain,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,ASA_188B,ASA_188,NA,NA,NA,NA,NA,NA,Brain,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
330,ASA_189A,ASA_189,NA,NA,NA,NA,NA,NA,Brain,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
331,ASA_189B,ASA_189,NA,NA,NA,NA,NA,NA,Brain,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
332,ASA_190A,ASA_190,NA,NA,NA,NA,NA,NA,Brain,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [45]:

subject_id = dfs["SUBJECT"]['subject_id']
primary_diagnosis = dfs["SUBJECT"]['primary_diagnosis'].str.lower().str.replace(" ", "_")
diagnosis_mapper = dict(zip(subject_id, primary_diagnosis))
SAMPLE_['condition_id'] = dfs["SAMPLE"]['subject_id'].map(diagnosis_mapper)

SAMPLE_['organism'] = dfs["SAMPLE"]['subject_id'].map(dict(zip(dfs["SUBJECT"]['subject_id'], dfs["SUBJECT"]['organism'])))


sample_subj_map = dict(zip(SAMPLE_['sample_id'], SAMPLE_['subject_id']))

In [63]:

# DATA
DATA_ = filter_table_columns(dfs["DATA"], CDEv3, "DATA")
dfs["DATA"]


,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,adjustment,...,technology_full_name,subject_ids,source_subject_ids,header,annotation,configuration_file,omic,tissue,input_cell_count,RIN
0,MO001a-RNA,Rep1,1,0,1,fastq,gs://asap-raw-data-team-voet/aertslab/rna/fast...,snMultiome RNA R1 FASTQ file,ffa6f9f887824815c8ed73e48146944b,raw,...,snMultiome RNA,ASA_014B|ASA_015B,SD002/17|SD048/16,NA,NA,NA,NA,NA,NA,NA
1,MO001a-RNA,Rep1,1,0,1,fastq,gs://asap-raw-data-team-voet/aertslab/rna/fast...,snMultiome RNA R2 FASTQ file,af3617a1d0865b7c19f7908dbdcff894,raw,...,snMultiome RNA,ASA_014B|ASA_015B,SD002/17|SD048/16,NA,NA,NA,NA,NA,NA,NA
2,MO001a-RNA,Rep1,1,0,1,fastq,gs://asap-raw-data-team-voet/aertslab/rna/fast...,snMultiome RNA R1 FASTQ file,eb75ad41e14c96fa19335faa359db128,raw,...,snMultiome RNA,ASA_014B|ASA_015B,SD002/17|SD048/16,NA,NA,NA,NA,NA,NA,NA
3,MO001a-RNA,Rep1,1,0,1,fastq,gs://asap-raw-data-team-voet/aertslab/rna/fast...,snMultiome RNA R2 FASTQ file,aa0b5bdedf96850d0efb058a9ca91548,raw,...,snMultiome RNA,ASA_014B|ASA_015B,SD002/17|SD048/16,NA,NA,NA,NA,NA,NA,NA
4,MO001a-RNA,Rep1,1,0,1,fastq,gs://asap-raw-data-team-voet/aertslab/rna/fast...,snMultiome RNA R1 FASTQ file,21701262f7c29f18b57ff06a552a86b1,raw,...,snMultiome RNA,ASA_014B|ASA_015B,SD002/17|SD048/16,NA,NA,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,RN006a,Rep1,1,0,1,fastq,gs://asap-raw-data-team-voet/aertslab/rna/fast...,snRNA R2 FASTQ file,b5913bea2d29660cac3b44ffca82c3e1,raw,...,snRNA,ASA_131A|ASA_132A|ASA_133A|ASA_134A|ASA_135A|A...,P29/12|P35/12|P22/13|P17/16|P11/17|P21/17|P29/...,NA,NA,NA,NA,NA,NA,NA
854,RN006a,Rep1,1,0,1,fastq,gs://asap-raw-data-team-voet/aertslab/rna/fast...,snRNA R1 FASTQ file,ca5f10a3981ef24bc836e30097ba25b8,raw,...,snRNA,ASA_131A|ASA_132A|ASA_133A|ASA_134A|ASA_135A|A...,P29/12|P35/12|P22/13|P17/16|P11/17|P21/17|P29/...,NA,NA,NA,NA,NA,NA,NA
855,RN006a,Rep1,1,0,1,fastq,gs://asap-raw-data-team-voet/aertslab/rna/fast...,snRNA R2 FASTQ file,da09c2152085bc7723fedaf916fce018,raw,...,snRNA,ASA_131A|ASA_132A|ASA_133A|ASA_134A|ASA_135A|A...,P29/12|P35/12|P22/13|P17/16|P11/17|P21/17|P29/...,NA,NA,NA,NA,NA,NA,NA
856,RN006a,Rep1,1,0,1,fastq,gs://asap-raw-data-team-voet/aertslab/rna/fast...,snRNA R1 FASTQ file,fc222b538277c31d56eb964d391ba1d7,raw,...,snRNA,ASA_131A|ASA_132A|ASA_133A|ASA_134A|ASA_135A|A...,P29/12|P35/12|P22/13|P17/16|P11/17|P21/17|P29/...,NA,NA,NA,NA,NA,NA,NA


In [60]:

# ASSAY_RNAseq
ASSAY_RNAseq_ = filter_table_columns(dfs["DATA"], CDEv3, "ASSAY_RNAseq")

# now get the reest from the columns from the DATA table
# hemisphere, region_level_1, region_level_2, region_level_3, pm_PH
# ASSAY_RNAseq[['sample_id', 'hemisphere', 'tissue', 'region_level_1', 'region_level_2',
#        'region_level_3', 'RIN', 'pm_PH', 'technology', 'omic',
#        'molecular_source', 'input_cell_count', 'assay', 'sequencing_end',

ASSAY_RNAseq_
#        'sequencing_length', 'sequencing_instrument', 'brain_region']]

,sample_id,tissue,technology,omic,RIN,molecular_source,input_cell_count,assay,sequencing_end,sequencing_length,sequencing_instrument
0,MO001a-RNA,NA,SN,NA,NA,PolyA RNA,NA,ARC-v1,Paired-end,28,Illumina NextSeq 500
1,MO001a-RNA,NA,SN,NA,NA,PolyA RNA,NA,ARC-v1,Paired-end,44,Illumina NextSeq 500
2,MO001b-RNA,NA,SN,NA,NA,PolyA RNA,NA,ARC-v1,Paired-end,28,Illumina NextSeq 500
3,MO001b-RNA,NA,SN,NA,NA,PolyA RNA,NA,ARC-v1,Paired-end,44,Illumina NextSeq 500
4,MO001b-RNA,NA,SN,NA,NA,PolyA RNA,NA,ARC-v1,Paired-end,28,Illumina NextSeq 2000
...,...,...,...,...,...,...,...,...,...,...,...
323,RN005e,NA,SN,NA,NA,PolyA RNA,NA,v3.1 - Single Index,Paired-end,90,Illumina NovaSeq 6000
324,RN006a,NA,SN,NA,NA,PolyA RNA,NA,v3.1 - Single Index,Paired-end,28,Illumina NextSeq 2000
325,RN006a,NA,SN,NA,NA,PolyA RNA,NA,v3.1 - Single Index,Paired-end,89,Illumina NextSeq 2000
326,RN006a,NA,SN,NA,NA,PolyA RNA,NA,v3.1 - Single Index,Paired-end,28,Illumina NovaSeq 6000


In [49]:

# PMDBS
PMDBS_= filter_table_columns(dfs["ASSAY_RNAseq"], CDEv3, "PMDBS")
dfs["ASSAY_RNAseq"]['subject_id', 'brain_region', 'hemisphere', 'region_level_1', 'region_level_2', 'region_level_3']


KeyError: ('subject_id', 'brain_region', 'hemisphere', 'region_level_1', 'region_level_2', 'region_level_3')

In [ ]:
technolog, omics, sequencing_end, sequencing_length, sequencing_instrument, molecular_source, assay

In [ ]:


ASSAY_RNAseqv3['technology'] = dfs["DATA"]['technology'][0]
ASSAY_RNAseqv3['omic'] = dfs["DATA"]['omic'][0]
v3_tables["ASSAY_RNAseq"] = ASSAY_RNAseqv3

# CLINPATH
CLINPATHv3 = filter_table_columns(v2_tables["CLINPATH"], CDEv3, "CLINPATH")
# move these to CLINPATHv3 from SUBJECTv2
cols_from_SUBJECTv2 = [
    'AMPPD_id',
    'GP2_id',
    'ethnicity',
    'family_history',
    'last_diagnosis',
    'age_at_onset',
    'age_at_diagnosis',
    'first_motor_symptom',
    'hx_dementia_mci',
    'hx_melanoma',
    'education_level',
    'smoking_status',
    'smoking_years',
    'APOE_e4_status',
    'cognitive_status',
    'time_from_baseline']
CLINPATHv3_cols = [col for col in CLINPATHv3.columns if col not in cols_from_SUBJECTv2]
cols_from_SUBJECTv2 += ['subject_id']
v3_tables["CLINPATH"] = pd.merge(CLINPATHv3[CLINPATHv3_cols], v2_tables["SUBJECT"][cols_from_SUBJECTv2], on="subject_id", how="left")

# CONDITION
# construct this table.  needs to be checked by hand
CONDITIONv3 = pd.DataFrame(columns=CDEv3[CDEv3['Table'] == "CONDITION"]['Field'])
CONDITIONv3['condition_id'] = v3_tables["SUBJECT"]['primary_diagnosis'].unique()
CONDITIONv3['intervention_name'] = "Case-Control"
CONDITIONv3['intervention_id'] = CONDITIONv3['condition_id'].apply(intervention_typer)
CONDITIONv3['condition_id'] = CONDITIONv3['condition_id'].str.lower().str.replace(" ", "_")

CONDITIONv3 = CONDITIONv3.fillna(NULL)

v3_tables["CONDITION"] = CONDITIONv3

## Validate and clean the metadata tables

In [3]:
metadata_version = "v2"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"


## convert 
data_path = Path.home() / ("Projects/ASAP/team-sulzer")
metadata_path = data_path / "metadata_email"


SUBJECT = read_meta_table(metadata_path / "SUBJECT.csv", dtypes_dict)
CLINPATH = read_meta_table(metadata_path / "CLINPATH.csv", dtypes_dict)
STUDY = read_meta_table(metadata_path / "STUDY.csv", dtypes_dict)
PROTOCOL = read_meta_table(metadata_path / "PROTOCOL.csv", dtypes_dict)
SAMPLE = read_meta_table(metadata_path / "SAMPLE.csv", dtypes_dict)
DATA = read_meta_table(metadata_path / "DATA.csv", dtypes_dict)

# SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv")
# CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv")
# STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv")
# PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv")
# SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv")
# DATA = pd.read_csv(f"{metadata_path}/DATA.csv")


read url


In [4]:

STUDY = prep_table(STUDY, CDE_df)
# update metadata_version_date
STUDY['metadata_version_date'] = METADATA_VERSION_DATE

In [5]:
report = ReportCollector(destination="")
validate_table(STUDY, "STUDY", CDE_df, report)
report.print_log()

All required fields are present in *STUDY* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in STUDY. 🥳



In [6]:
# fix the column order
PROTOCOL = prep_table(PROTOCOL, CDE_df)
report = ReportCollector(destination="")
validate_table(PROTOCOL, "PROTOCOL", CDE_df, report)
report.print_log()



All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



In [7]:

SUBJECT = prep_table(SUBJECT, CDE_df)



SUBJECT['sex'].replace({"M":"Male", "F":"Female"}, inplace=True)


In [8]:

SUBJECT['hx_dementia_mci'].replace("Unknown", pd.NA, inplace=True)
SUBJECT['hx_melanoma'].replace("Unknown", pd.NA, inplace=True)
SUBJECT['education_level'].replace("Unknown", pd.NA, inplace=True)
SUBJECT['cognitive_status'].replace("Unknown", pd.NA, inplace=True)
SUBJECT['primary_diagnosis'].replace("Unknown", pd.NA, inplace=True)


In [9]:
report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()

All required fields are present in *SUBJECT* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enums
🚨⚠️❗ **Invalid entries**
- _*hx_dementia_mci*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*hx_melanoma*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*education_level*_:  invalid values 💩'<NA>'
    - valid ➡️ 'High School', 'High School/GED', 'Some college without degree', 'Associate degree college', 'Bachelor's degree', 'Master's degree', 'Professional or doctoral degree', 'Refuse', 'Other'
- _*cognitive_status*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Normal', 'MCI', 'Dementia'
- _*primary_diagnosis*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Healthy Control', 'Idiopathic PD', 'Alzheimer’s disease', 'Frontotemporal dementia', 'Corticobasal syndrome', 'Dementia with Lewy bodies', 'Dopa-responsive dystonia', 'Essential tremor', 'Hemiparkinson/hemiatrophy syndrome', 'Juvenile autosomal recessive parkinson

In [10]:

SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"
SAMPLE['molecular_source'].replace("RNA", "PolyA RNA", inplace=True)
SAMPLE['region_level_2'].replace('Substantia Nigra', 'Substantia nigra', inplace=True)

In [11]:
batch_mapper = {"Cingulate Cortex":"cing1", "Substantia Nigra": "sn1"}
SAMPLE['batch']=SAMPLE['brain_region'].map(batch_mapper)

In [12]:
SAMPLE = prep_table(SAMPLE, CDE_df)
report = ReportCollector(destination="")
validate_table(SAMPLE, "SAMPLE", CDE_df, report)
report.print_log()

All required fields are present in *SAMPLE* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- RIN: 28/46 empty rows

	- source_RIN: 46/46 empty rows

	- input_cell_count: 28/46 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- DV200: 46/46 empty rows

	- pm_PH: 46/46 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*hemisphere*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Right', 'Left', 'Unknown'



In [13]:
# CLINPATH.fillna("Nan")
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()



All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- path_year_death: 46/46 empty rows

	- brain_weight: 46/46 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enums
🚨⚠️❗ **Invalid entries**
- _*path_braak_nft*_:  invalid values 💩'<NA>'
    - valid ➡️ '0', 'I', 'II', 'III', 'IV', 'V', 'VI', 'I/II', 'III/IV', 'IV/V', 'V/VI'
- _*path_cerad*_:  invalid values 💩'<NA>'
    - valid ➡️ 'None', 'Sparse', 'Moderate', 'Frequent'
- _*path_thal*_:  invalid values 💩'<NA>'
    - valid ➡️ '0', '1', '2', '3', '4', '5', '1/2', '3', '4/5'
- _*known_pathogenic_mutation*_:  invalid values 💩'<NA>'
    - valid ➡️ 'None', 'Present', 'Unknown'
- _*path_mckeith*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Neocortical', 'Limbic (transitional)', 'Brainstem', 'Amygdala Predominant', 'Olfactory bulb only', 'Limbic, transitional (brainstem and limbic involvement)', 'Diffuse, neocortical (brainstem, limbic and neocortical involvement)', 'Olfactory B

In [14]:
CLINPATH = prep_table(CLINPATH, CDE_df)
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()


All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- path_year_death: 46/46 empty rows

	- brain_weight: 46/46 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enums
🚨⚠️❗ **Invalid entries**
- _*path_braak_nft*_:  invalid values 💩'<NA>'
    - valid ➡️ '0', 'I', 'II', 'III', 'IV', 'V', 'VI', 'I/II', 'III/IV', 'IV/V', 'V/VI'
- _*path_cerad*_:  invalid values 💩'<NA>'
    - valid ➡️ 'None', 'Sparse', 'Moderate', 'Frequent'
- _*path_thal*_:  invalid values 💩'<NA>'
    - valid ➡️ '0', '1', '2', '3', '4', '5', '1/2', '3', '4/5'
- _*known_pathogenic_mutation*_:  invalid values 💩'<NA>'
    - valid ➡️ 'None', 'Present', 'Unknown'
- _*path_mckeith*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Neocortical', 'Limbic (transitional)', 'Brainstem', 'Amygdala Predominant', 'Olfactory bulb only', 'Limbic, transitional (brainstem and limbic involvement)', 'Diffuse, neocortical (brainstem, limbic and neocortical involvement)', 'Olfactory B

In [15]:
DATA.head()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,technology,omic,adjustment,content,time,header,annotation,configuration_file
0,T-362,rep1,1,0,1,fastq,<NA>,<NA>,<NA>,sN,RNA,Raw,Counts,NaN,<NA>,<NA>,<NA>
1,T-308,rep1,1,0,1,fastq,<NA>,<NA>,<NA>,sN,RNA,Raw,Counts,NaN,<NA>,<NA>,<NA>
2,T3860,rep1,1,0,1,fastq,<NA>,<NA>,<NA>,sN,RNA,Raw,Counts,NaN,<NA>,<NA>,<NA>
3,T290,rep1,1,0,1,fastq,<NA>,<NA>,<NA>,sN,RNA,Raw,Counts,NaN,<NA>,<NA>,<NA>
4,T-233,rep1,1,0,1,fastq,<NA>,<NA>,<NA>,sN,RNA,Raw,Counts,NaN,<NA>,<NA>,<NA>


## 🚨⚠️❗ EXTREME CURATION HERE 🚨⚠️❗
The codeing by Team-Sulzer is inconsistent.  However I am able to infer the following:

TODO:  Define `sample_id`s from `subject_id`, and create `sample_id` as `source_subject_id`


`source_subject_id`s seem to be unique and correct.  Although the CLINPATH.duration_pmi are different for `source_subject_id`="T-2465", and some demographic info is missing for the cingulate sample, these seem to be two samples from the same subject.  

Hence:  We will assume `source_subject_id` is ground truth.   Redefine `sample_id` from the current `subject_id` (which matches file naming conventsions), and  create a _new_ `subject_id` from the `source_subject_id` by prepending "SUL_".  Finally, `source_sample_id` will be set to pd.NA as is currently just a copy of `source_subject_id`

### fix SAMPLE table first

In [16]:
# simple copy subject_id to sample_id
SAMPLE['sample_id'] = SAMPLE['subject_id']


In [17]:

# then make a subject_id by copying souce_sample_id
SAMPLE['subject_id'] = "SUL_" + SAMPLE['source_sample_id']

# finally set source_sample_id to pd.NA
SAMPLE['source_sample_id'] = pd.NA

### now SUBJECT table 

In [18]:
SUBJECT['subject_id'] = "SUL_" + SUBJECT['source_subject_id']

### and CLINPATH table is already coded as it should be



## DATA table

The DATA.csv was not filled out.  So we will construct it by joining the manifest from the source of data transfer for Team Sulzer.

### read_file_list
Helper function to read sulzer manifest 

In [19]:

cingulate_files = Path.home() / "Projects/ASAP/team-sulzer/cingulate_manifest.txt"
nigra_files = Path.home() / "Projects/ASAP/team-sulzer/nigra_manifest.txt"

In [20]:
# tmp2 = pd.read_csv("sulzer_fastqs.txt", delimiter=" ", header=None, names=['file_name'])
def read_gz_file_list(file_nm):
    # Read the text file
    with open(file_nm, 'r') as file:
        lines = [line.strip() for line in file]

    # Create a DataFrame
    df = pd.DataFrame(lines, columns=['file_name'])

    df['parts'] = df['file_name'].apply(lambda x: x.lstrip("gs://").split("/"))
    df['source'] = df['parts'].apply(lambda x: x[1].lstrip("fastqs_") )
    df['name'] = df['parts'].apply(lambda x: x[-1])

    df['name_parts'] = df['name'].apply(lambda x: x.rstrip('.fastq.gz').split("_"))
    df['sample_id'] = df['name_parts'].apply(lambda x: x[0])
    df['lane_id'] = df['name_parts'].apply(lambda x: x[2])
    df['file_type'] = df['name_parts'].apply(lambda x: x[3])
    df.drop(columns=['parts','name_parts'], inplace=True)
    return df

# tmp2 = pd.read_csv("sulzer_fastqs.txt", delimiter=" ", header=None, names=['file_name'])
def read_file_list_sulzer(file_nm):
    # Read the text file
    keep_ends = ("fastq.gz", "bam", "tar")
    with open(file_nm, 'r') as file:
        lines = [line.strip() for line in file if line.strip().endswith(keep_ends)]

    # Create a DataFrame
    df = pd.DataFrame(lines, columns=['file_name'])

    df['parts'] = df['file_name'].apply(lambda x: x.split("/"))
    # df['source'] = df['parts'].apply(lambda x: x[5].lstrip("fastqs_") )
    df['file_name'] = df['parts'].apply(lambda x: x[-1])
    df['sample_id'] = df['parts'].apply(lambda x: x[6] if x[-1].endswith(("fastq.gz","tar")) else x[7])
    df['file_type'] = df['parts'].apply(lambda x: x[7] if x[7].strip() in ["fastq","analysis"] else "bam")
    # df['tmp1'] = df['parts'].apply(lambda x: x[7] if len(x) > 7 else pd.NA)

    # df['name_parts'] = df['name'].apply(lambda x: x.rstrip('.fastq.gz').split("_"))
    # df['sample_id'] = df['name_parts'].apply(lambda x: x[0])
    # df['lane_id'] = df['name_parts'].apply(lambda x: x[2])
    # df['file_type'] = df['name_parts'].apply(lambda x: x[3])
    df.drop(columns=['parts'], inplace=True)
    return df




In [21]:

c_fastqs = read_file_list_sulzer(cingulate_files)
c_fastqs['src'] = "cingulate"
n_fastqs = read_file_list_sulzer(nigra_files)
n_fastqs['src'] = "nigra"

from_source = pd.concat([c_fastqs, n_fastqs])

In [22]:
n_fastqs

,file_name,sample_id,file_type,src
0,OJ1_cellranger_count_outs.tar,OJ1,analysis,nigra
1,OJ1_S1_L001_I1_001.fastq.gz,OJ1,fastq,nigra
2,OJ1_S1_L001_I2_001.fastq.gz,OJ1,fastq,nigra
3,OJ1_S1_L001_R1_001.fastq.gz,OJ1,fastq,nigra
4,OJ1_S1_L001_R2_001.fastq.gz,OJ1,fastq,nigra
...,...,...,...,...
454,OJ9_S1_L003_R2_001.fastq.gz,OJ9,fastq,nigra
455,OJ9_S1_L004_I1_001.fastq.gz,OJ9,fastq,nigra
456,OJ9_S1_L004_I2_001.fastq.gz,OJ9,fastq,nigra
457,OJ9_S1_L004_R1_001.fastq.gz,OJ9,fastq,nigra


In [23]:
## now make DATA from scratch to ready to join into
# Start with sample_id
DATA_ = read_meta_table(metadata_path / "DATA.csv", dtypes_dict)

DATA = SAMPLE[["sample_id", "replicate", "replicate_count","repeated_sample","batch"]].copy()

DATA.head()



,sample_id,replicate,replicate_count,repeated_sample,batch
0,OF001,Rep1,1,0,cing1
1,OF009,Rep1,1,0,cing1
2,OF016,Rep1,1,0,cing1
3,OF005,Rep1,1,0,cing1
4,OF013,Rep1,1,0,cing1


In [24]:
DATA = pd.merge(DATA, from_source, on='sample_id', how='left')
DATA.tail()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_name,file_type,src
707,OJ26,Rep1,1,0,sn1,OJ26_S1_L003_R2_001.fastq.gz,fastq,nigra
708,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_I1_001.fastq.gz,fastq,nigra
709,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_I2_001.fastq.gz,fastq,nigra
710,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_R1_001.fastq.gz,fastq,nigra
711,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_R2_001.fastq.gz,fastq,nigra


In [25]:
DATA['omic'] = 'RNA'
DATA['adjustments'] = 'Raw'
DATA['technology'] = "SN"
DATA['file_description']= DATA['file_type'].map({"bam": "Binary aligned sequence",
                                                "fastq": "gzip fastq",
                                                "analysis": "cellranger counts tar archive"})
DATA['file_description']= DATA['file_type'].map({"bam": "Reads",
                                                "fastq": "Reads",
                                                "analysis": "Counts"})

# now fix the file_type
DATA['file_type'] = DATA['file_type'].replace({'fastq':'fastq', "bam":'Per sample raw file', "analysis":'Per sample processed file'})
DATA['content'] = 'Counts'



get "md5s"

In [26]:


cingulate_md5s = Path.home() / "Projects/ASAP/team-sulzer/cingulate_fastq_md5.log"
nigra_md5s = Path.home() / "Projects/ASAP/team-sulzer/nigra_fastq_md5.log"

In [27]:
# cingulate
cingulate_hashs = extract_hashes_from_gsutil(cingulate_md5s)

# nigra
nigra_hashs = extract_hashes_from_gsutil(nigra_md5s)

# combine cingulate and nigra
source_file_crc = cingulate_hashs[0] | nigra_hashs[0]
source_file_md5 = cingulate_hashs[1] | nigra_hashs[1]


In [28]:
DATA['file_MD5'] = DATA['file_name'].map(source_file_md5)

In [29]:

DATA = reorder_table_to_CDE(DATA,"DATA", CDE_df)
DATA.head()



added empty column adjustment to DATA table
added empty column time to DATA table
added empty column header to DATA table
added empty column annotation to DATA table
added empty column configuration_file to DATA table


,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,technology,omic,adjustment,content,time,header,annotation,configuration_file
0,OF001,Rep1,1,0,cing1,Per sample raw file,possorted_genome_bam.bam,Reads,NaN,SN,RNA,<NA>,Counts,<NA>,<NA>,<NA>,<NA>
1,OF001,Rep1,1,0,cing1,Per sample processed file,OF001_cellranger_count_outs.tar,Counts,NaN,SN,RNA,<NA>,Counts,<NA>,<NA>,<NA>,<NA>
2,OF001,Rep1,1,0,cing1,fastq,OF001_S1_L001_I1_001.fastq.gz,Reads,74a74c8929b01a6da7e79b83f91a6607,SN,RNA,<NA>,Counts,<NA>,<NA>,<NA>,<NA>
3,OF001,Rep1,1,0,cing1,fastq,OF001_S1_L001_I2_001.fastq.gz,Reads,644d95728b50f1ff9db7becfa5b5c394,SN,RNA,<NA>,Counts,<NA>,<NA>,<NA>,<NA>
4,OF001,Rep1,1,0,cing1,fastq,OF001_S1_L001_R1_001.fastq.gz,Reads,129dfb6fe6115966e2cf2af5e3c82bdf,SN,RNA,<NA>,Counts,<NA>,<NA>,<NA>,<NA>


In [30]:

report = ReportCollector(destination="")
validate_table(DATA, "DATA", CDE_df, report)
report.print_log()


All required fields are present in *DATA* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- time: 712/712 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enums
🚨⚠️❗ **Invalid entries**
- _*file_type*_:  invalid values 💩'nan'
    - valid ➡️ 'fastq', 'Per sample raw file', 'Per sample processed file', 'Combined analysis files', 'annData', 'vHDF', 'plink2', 'VCF', 'csv'
- _*adjustment*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Raw', 'Processed'



# No data found for `OJ8`!!!

# load manifests and md5s

Separate manifests for cingulate and nigra to make it easier for Team Sulzer to collect.


In [31]:
SAMPLE[SAMPLE['sample_id']=='OJ8']
empty_sample = SAMPLE['sample_id']=='OJ8'
print(SAMPLE.shape)
SAMPLE = SAMPLE[~empty_sample]
SAMPLE.shape

(46, 33)


(45, 33)

In [32]:
DATA[DATA['sample_id']=='OJ8']
empty_sample = DATA['sample_id']=='OJ8'
print(DATA.shape)
DATA = DATA[~empty_sample]
DATA.shape

(712, 17)


(711, 17)

## check remote files



In [31]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/sulzer-credentials.json 


Activated service account credentials for: [raw-admin-sulzer@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [29]:

!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/**/*.gz"


   4249181  2023-12-01T21:26:39Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_info.tar.gz#1701465999745881  metageneration=1
 752394240  2023-12-02T00:16:13Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I1_001.fastq.gz#1701476173757016  metageneration=1
 757577390  2023-12-02T00:24:37Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I2_001.fastq.gz#1701476677261783  metageneration=1
7398635734  2023-12-02T18:37:54Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_R1_001.fastq.gz#1701542274674731  metageneration=1
5587519431  2023-12-02T15:13:17Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/fastq/SN_3PGEX_10X/210210_A00521_019

## fix missing file_name, file_description, file_MD5 

Collect "destination" or bucket md5s and manifest

In [74]:

!gsutil -u dnastack-asap-parkinsons ls "gs://asap-raw-data-team-sulzer/**/*fastq.gz" > sulzer_bucket_fasqs.txt

In [76]:
!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-sulzer/**/*.gz" > sulzer_bucket_hexhash.log


using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [96]:
# bucket_fastqs = pd.read_csv("sulzer_bucket_fasqs.txt", delimiter="  ", header=None, names=['file_name'])
# bucket_fastqs

In [33]:
bucket_files_md5 = extract_md5_from_details2("sulzer_bucket_hexhash.log")


In [34]:
tmp = pd.DataFrame.from_dict(bucket_files_md5, orient="index",columns=['md5'])
tmp['file'] = tmp.index
tmp = tmp.sort_values(by='file').reset_index()

tmp


,index,md5,file
0,OF001_S1_L001_I1_001.fastq.gz,74a74c8929b01a6da7e79b83f91a6607,OF001_S1_L001_I1_001.fastq.gz
1,OF001_S1_L001_I2_001.fastq.gz,644d95728b50f1ff9db7becfa5b5c394,OF001_S1_L001_I2_001.fastq.gz
2,OF001_S1_L001_R1_001.fastq.gz,129dfb6fe6115966e2cf2af5e3c82bdf,OF001_S1_L001_R1_001.fastq.gz
3,OF001_S1_L001_R2_001.fastq.gz,dbdec2b83d4564e67284a51c8d1f5dc2,OF001_S1_L001_R2_001.fastq.gz
4,OF001_S1_L002_I1_001.fastq.gz,50d1c4be0b4c65310fac646fd676f3c4,OF001_S1_L002_I1_001.fastq.gz
...,...,...,...
724,OJ9_S1_L004_I1_001.fastq.gz,2dba0b38f76ba6490260266e02689392,OJ9_S1_L004_I1_001.fastq.gz
725,OJ9_S1_L004_I2_001.fastq.gz,ed6433715131d4ea289a5644bb89ce0b,OJ9_S1_L004_I2_001.fastq.gz
726,OJ9_S1_L004_R1_001.fastq.gz,a6ec79d470af33aab7e72c6703ea26fa,OJ9_S1_L004_R1_001.fastq.gz
727,OJ9_S1_L004_R2_001.fastq.gz,a9cb6810affb5ae38115d980048c31f9,OJ9_S1_L004_R2_001.fastq.gz


## Only check the fastq.gz files.  don't have md5s for the others

In [35]:
DATA_ = DATA[DATA['file_type']=="fastq"]

In [36]:



checksum = DATA_[['file_name','file_MD5']]
checksum['check1'] = checksum['file_MD5'].str.strip()
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)



/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_40441/2423319654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check1'] = checksum['file_MD5'].str.strip()
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_40441/2423319654.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check2'] = checksum['file_name'].map(bucket_files_md5)


In [37]:
checksum[checksum.check1 != checksum.check2].file_name.to_list()


[]

In [38]:
checksum.head()

,file_name,file_MD5,check1,check2
2,OF001_S1_L001_I1_001.fastq.gz,74a74c8929b01a6da7e79b83f91a6607,74a74c8929b01a6da7e79b83f91a6607,74a74c8929b01a6da7e79b83f91a6607
3,OF001_S1_L001_I2_001.fastq.gz,644d95728b50f1ff9db7becfa5b5c394,644d95728b50f1ff9db7becfa5b5c394,644d95728b50f1ff9db7becfa5b5c394
4,OF001_S1_L001_R1_001.fastq.gz,129dfb6fe6115966e2cf2af5e3c82bdf,129dfb6fe6115966e2cf2af5e3c82bdf,129dfb6fe6115966e2cf2af5e3c82bdf
5,OF001_S1_L001_R2_001.fastq.gz,dbdec2b83d4564e67284a51c8d1f5dc2,dbdec2b83d4564e67284a51c8d1f5dc2,dbdec2b83d4564e67284a51c8d1f5dc2
6,OF001_S1_L002_I1_001.fastq.gz,50d1c4be0b4c65310fac646fd676f3c4,50d1c4be0b4c65310fac646fd676f3c4,50d1c4be0b4c65310fac646fd676f3c4


## Sanitize NULLs

In [41]:

CLINPATH = sanitize(CLINPATH)
PROTOCOL = sanitize(PROTOCOL)
SUBJECT = sanitize(SUBJECT)
STUDY = sanitize(STUDY)
DATA = sanitize(DATA)
SAMPLE = sanitize(SAMPLE)

## Export to clean

In [42]:

export_root = Path.cwd() / "clean/team-Sulzer"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)

STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")
DATA.to_csv(export_root / "DATA.csv")


## transfer metadata Sulzer 

In [43]:
# Sulzer
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/sulzer-credentials.json 


Activated service account credentials for: [raw-admin-sulzer@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [44]:

!gsutil -u dnastack-asap-parkinsons cp "./clean/team-sulzer/*.csv"  "gs://asap-raw-data-team-sulzer/metadata/v2/"

Copying file://./clean/team-sulzer/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-sulzer/SUBJECT.csv [Content-Type=text/csv]...       
Copying file://./clean/team-sulzer/SAMPLE.csv [Content-Type=text/csv]...        
Copying file://./clean/team-sulzer/DATA.csv [Content-Type=text/csv]...          
- [4 files][107.8 KiB/107.8 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-sulzer/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-sulzer/PROTOCOL.csv [Content-Type=text/csv]...      
\ [6 files][109.1 KiB/109.1 KiB]                                                
Operation completed over 6 objects/109.1 KiB.                                    


### copy metadata to workflow-dev bucket

First copy each set of metadata locally ...

In [65]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/wf-credentials.json

Activated service account credentials for: [admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [66]:
!gsutil cp "./clean/team-Sulzer/*.csv" "gs://asap-workflow-dev/metadata/v2/sulzer"

Copying file://./clean/team-Sulzer/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Sulzer/SUBJECT.csv [Content-Type=text/csv]...       
Copying file://./clean/team-Sulzer/SAMPLE.csv [Content-Type=text/csv]...        
Copying file://./clean/team-Sulzer/DATA.csv [Content-Type=text/csv]...          
- [4 files][112.1 KiB/112.1 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Sulzer/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Sulzer/PROTOCOL.csv [Content-Type=text/csv]...      
\ [6 files][113.5 KiB/113.5 KiB]                                                
Operation completed over 6 objects/113.5 KiB.                                    


In [67]:
!gsutil ls -al "gs://asap-workflow-dev/metadata/v2"

                                 gs://asap-workflow-dev/metadata/v2/hafler/
                                 gs://asap-workflow-dev/metadata/v2/hardy/
                                 gs://asap-workflow-dev/metadata/v2/jakobsson/
                                 gs://asap-workflow-dev/metadata/v2/lee/
                                 gs://asap-workflow-dev/metadata/v2/scherzer/
                                 gs://asap-workflow-dev/metadata/v2/sulzer/


In [312]:

# move to cannonical location
!gsutil -u dnastack-asap-parkinsons cp -r "gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated" "gs://asap-raw-data-team-sulzer/fastqs/cingulate"


Copying gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_info.tar.gz [Content-Type=application/x-tar]...
Copying gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_outs.tar [Content-Type=application/x-tar]...
Copying gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/README.txt [Content-Type=text/plain]...
Copying gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/index.html [Content-Type=text/html]...
- [4 files][  1.2 GiB/  1.2 GiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Pl

In [317]:

# move to cannonical location
!gsutil -u dnastack-asap-parkinsons mv -r "gs://asap-raw-data-team-sulzer/fastqs_nigra" "gs://asap-raw-data-team-sulzer/fastqs/nigra"


Copying gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/OJ1_cellranger_count_info.tar.gz [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/OJ1_cellranger_count_info.tar.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/OJ1_cellranger_count_outs.tar [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/OJ1_cellranger_count_outs.tar...
Copying gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/README.txt [Content-Type=text/plain]...
Removing gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger

In [350]:

!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-sulzer/fastqs/cingulate/**/*.fastq.gz" "gs://asap-raw-data-team-sulzer/fastqs/cingulate/"


Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I1_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I2_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_R1_001.fastq.gz...
Copying gs://

In [353]:
!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-sulzer/fastqs/nigra"


 342559148  2023-12-06T00:19:01Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L001_I1_001.fastq.gz#1701821941986375  metageneration=1
 359917684  2023-12-06T00:19:02Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L001_I2_001.fastq.gz#1701821942302756  metageneration=1
3431481573  2023-12-06T00:19:02Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L001_R1_001.fastq.gz#1701821942601943  metageneration=1
2572117362  2023-12-06T00:19:02Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L001_R2_001.fastq.gz#1701821942905242  metageneration=1
 341079281  2023-12-06T00:19:03Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L002_I1_001.fastq.gz#1701821943223278  metageneration=1
 359720996  2023-12-06T00:19:03Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L002_I2_001.fastq.gz#1701821943511357  metageneration=1
3410135611  2023-12-06T00:19:03Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L002_R1_001.fastq.gz#1701821943811147  metageneration=1
255816

In [352]:

!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-sulzer/fastqs/nigra/**/*.fastq.gz" "gs://asap-raw-data-team-sulzer/fastqs/nigra/"


Copying gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_I1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_I1_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_I2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_I2_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_R1_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq

In [341]:
!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files" "gs://asap-raw-data-team-sulzer/artifacts/bam_files"


Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF001/index.html [Content-Type=text/html]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF001/index.html...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF001/possorted_genome_bam.bam [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF001/possorted_genome_bam.bam...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF002/index.html [Content-Type=text/html]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF002/index.html...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF002/possorted_genome_bam.bam [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF002/possorted_genome_bam.bam...

==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil 

In [335]:
!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-sulzer/fastqs/**/analysis/*tar*" "gs://asap-raw-data-team-sulzer/artifacts/cellranger_counts"


Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_info.tar.gz [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_info.tar.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_outs.tar [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_outs.tar...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF002/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF002-cellranger-count-default/OF002_cellranger_count_info.tar.gz [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastq

## get artifacts

 "gs://asap-raw-data-team-sulzer/manifest.txt"

In [90]:
!gsutil -u dnastack-asap-parkinsons -m cp -r \
  "gs://asap-raw-data-team-sulzer/artifacts" \
  /Users/ergonyc/Projects/ASAP/team-sulzer/

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://asap-raw-data-team-sulzer/artifacts/cingulate_lineages_sublineages_assigned_dims18_k20_res02_final.rds...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://asap-raw-data-team-sulzer/artifacts/master_seurat_PD_filtered_doublet_removed_SCT_clusterslm3_res01_linege_called.rds...
Copying gs://asap-raw-data-team-sulzer/artifacts/nigra_cellbender.rds...        
Copying gs://asap-raw-data-team-sulzer/artifacts/nigra_obj

In [91]:
!gsutil -u dnastack-asap-parkinsons -m cp -r \
  "gs://asap-raw-data-team-sulzer/scripts" \
  /Users/ergonyc/Projects/ASAP/team-sulzer/

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://asap-raw-data-team-sulzer/scripts/cellbender_sample_processing.R...
/ [1/1 files][  1.9 KiB/  1.9 KiB] 100% Done                                    
Operation completed over 1 objects/1.9 KiB.                                      


In [ ]:
gsutil -m cp \
  "gs://asap-raw-data-team-sulzer/artifacts/cingulate_lineages_sublineages_assigned_dims18_k20_res02_final.rds" \
  "gs://asap-raw-data-team-sulzer/artifacts/master_seurat_PD_filtered_doublet_removed_SCT_clusterslm3_res01_linege_called.rds" \
  "gs://asap-raw-data-team-sulzer/artifacts/nigra_cellbender.rds" \
  "gs://asap-raw-data-team-sulzer/artifacts/nigra_obj_dims16_res02_k20_sct_harmony_dblrm_processed_final.rds" \
  .